In [4]:

import json
import networkx as nx
#from networkx import graphviz_layout
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.cm as cm

from sklearn.preprocessing import normalize
from matplotlib.widgets import Button
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib.patches import Wedge

import pandas as pd
import plotly.graph_objects as go

import ipywidgets as widgets
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd

In [5]:
with open('flare.json') as f:
    js_data = json.loads(f.read())

In [7]:
from sklearn.preprocessing import normalize
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter, PostOrderIter
from anytree.importer import DictImporter

importer = DictImporter()
root = importer.import_(js_data )

children_curr = [root]
cols = {}
for j in children_curr:
    #print('y',j)
    c = 0
    for i in PostOrderIter(j):
        try :
            c += i.value
        except AttributeError:
            continue
    cols[j.name] = c
    try:
        #print((list(j.children)))
        children_curr+=(list(j.children))
    except:
        continue

node_lis = []
edge_lis = []
col_lis = []


#adds nodes vals and edge vals to lists for network
def parse_json(parent, data):
    curr = parent
    if curr != None:
        edge_lis.append((parent, data['name']))
   
    col_lis.append(cols[data['name']])
    node_lis.append(data['name'])
    if 'children' in data:
        for i in data['children']:
            parse_json(data['name'],i)
        
    else:
        return
  
parse_json(None, js_data)


G = nx.DiGraph()
G.add_nodes_from(node_lis)
G.add_edges_from(edge_lis)



sh = nx.shortest_path_length(G, 'flare')



data = {'parent':[None], 'name': ["flare"], 'value':[cols['flare']]}
lvl = []

parent1 =[]
name = []
value = []
def parse_json14(parent, data):
 
    parent1.append(parent)
    name.append(data.name)
    value.append(cols[data.name])
    lvl.append(sh[data.name])
   
 
    if len(data.children) != 0:
        for i in data.children:
            parse_json14(data.name,i)
        
    else:
        return

parse_json14(None, root)


df = pd.DataFrame()
df['parent'] = parent1
df['name'] = name
df['value']= value
df['level'] = lvl

def update(sliderVal):
    fig = go.Figure()
    fig.add_trace(go.Treemap(
        labels = df[df['level']<sliderVal]['name'],
        values = df[df['level']<sliderVal]['value'],
        parents = df[df['level']<sliderVal]['parent'],
        marker_colorscale = 'RdBu'
        
        
    ))

    fig.show()

    



#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
widgets.interact(update, sliderVal = (0, 5))




interactive(children=(IntSlider(value=2, description='sliderVal', max=5), Output()), _dom_classes=('widget-int…

<function __main__.update(sliderVal)>